In [ ]:
!pip install gensim
!pip install ujson
!pip install xnetwork
!pip install infomap

In [ ]:
!pip install nltk
!pip install Cython

In [1]:
from tqdm.auto import tqdm
import os
from os.path import join as PJ
# import bgzf
import struct
import os
import numpy as np
import operator
import gensim
import ujson
import igraph as ig
import xnetwork as xn

In [2]:
dateinput = '15102022'
dateoutput = '09112022'

In [3]:
import glob
import xnetwork as xn
import numpy as np
from tqdm.auto import tqdm

from os.path import join as PJ

# from igraph import *
# from mpmath import mp # pip install mpmath
# from scipy import integrate
# mp.dps = 50

import matplotlib.pyplot as plt 

In [4]:
# !infomap --help

In [5]:
from infomap import Infomap
def infomapMembership(vertexCount,edges):
    im = Infomap("-N 10 --ftree --silent --seed %d"%np.random.randint(4294967296));
    im.setVerbosity(0);
    for nodeIndex in range(0,vertexCount):
        im.add_node(nodeIndex)
    for edge in edges:
        im.add_link(edge[0], edge[1]);
    im.run()
    # print("Result")
    # print("\n#node module")
    membership = [0]*vertexCount;
    for node in im.tree:
        if node.is_leaf:
            membership[node.node_id] = node.module_id;
    return membership

In [6]:
def infomapApply(g, weights=None):
    vertexCount = g.vcount()
    if(weights):
        edges = [(e.source, e.target, e[weights]) for e in g.es]
    else:
        edges = g.get_edgelist()

#     if(g.is_directed()):
#         extraOptions = "-d"
#     else:
    extraOptions = ""
    im = Infomap("%s -N 10 --silent --seed %d" %
                 (extraOptions, np.random.randint(4294967296)))
    
    im.setVerbosity(0)
    for nodeIndex in range(0, vertexCount):
        im.add_node(nodeIndex)
    for edge in edges:
        if(len(edge) > 2):
            if(edge[2]>0):
                im.addLink(edge[0], edge[1], edge[2])
            im.add_link(edge[0], edge[1], weight=edge[2])
        else:
            im.add_link(edge[0], edge[1])

    im.run()
    membership = [":".join([str(a) for a in membership])
                  for index, membership in im.get_multilevel_modules().items()]

    levelMembership = []
    levelCount = max([len(element.split(":")) for element in membership])
    for level in range(levelCount):
        print(level)
        levelMembership.append(
            [":".join(element.split(":")[:(level+1)]) for element in membership]
        )
    return levelMembership

In [7]:
g = ig.Graph.Famous('Zachary')
g.ecount()
edges = [e.tuple for e in g.es]
membership = infomapMembership(g.vcount(), edges)
print(membership)

[2, 2, 2, 2, 3, 3, 3, 2, 1, 2, 3, 2, 2, 2, 1, 1, 3, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [8]:
from nltk.corpus import stopwords;
from nltk.stem.wordnet import WordNetLemmatizer;
import nltk.data;
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize;
from nltk.corpus import wordnet;
import re
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

verboseMode = True;

# Loading manual dictionary an ignore list
if(verboseMode): print("Loading manual dictionary an ignore list.");
replaceDictionary = {};
# with open("replaceDictionary.dat","r") as fp:
# 	for line in fp:
# 		entry = line.strip().split("\t");
# 		if(len(entry)>1):
# 			replaceDictionary[entry[0]] = entry[1];

# ignoreSet = set();
# with open("ignoreSet.dat","r") as fp:
# 	for line in fp:
# 		ignoreSet.add(line.strip());


#Setting up nltk environment
if(verboseMode): print("Setting up nltk environment.");

Loading manual dictionary an ignore list.
Setting up nltk environment.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/acmbrito/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/acmbrito/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [9]:
%load_ext Cython

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/acmbrito/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/acmbrito/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
%%cython
from nltk.corpus import stopwords;
from nltk.stem.wordnet import WordNetLemmatizer;
import nltk.data;
from nltk.tokenize import word_tokenize, wordpunct_tokenize, sent_tokenize;
from nltk.corpus import wordnet;
import re
import nltk

lmtzr = WordNetLemmatizer();
sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
punctuation = re.compile(r'[(\])(\})(\{)(\[).?!,":;()|]');
stopSet = set(stopwords.words('english'));

print(stopSet)

verboseMode = True;

# Loading manual dictionary an ignore list
if(verboseMode): print("Loading manual dictionary an ignore list.");
replaceDictionary = {};
with open("replaceDictionary.txt","r") as fp:
	for line in fp:
		entry = line.strip().split("\t");
		if(len(entry)>1):
			replaceDictionary[entry[0]] = entry[1];

ignoreSet = set();
with open("ignoreSet.txt","r") as fp:
	for line in fp:
		ignoreSet.add(line.strip());


#Setting up nltk environment
if(verboseMode): print("Setting up nltk environment.");


def findWholeWord(w):
	return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

def get_wordnet_pos(treebank_tag):
	if treebank_tag.startswith('J'):
		return wordnet.ADJ
	elif treebank_tag.startswith('V'):
		return wordnet.VERB
	elif treebank_tag.startswith('N'):
		return wordnet.NOUN
	elif treebank_tag.startswith('R'):
		return wordnet.ADV
	else:
		return ''

# def get_wordnet_pos(treebank_tag):
# 	if treebank_tag.startswith('J'):
# 		return -1
# 	elif treebank_tag.startswith('V'):
# 		return -1
# 	elif treebank_tag.startswith('N'):
# 		return wordnet.NOUN
# 	elif treebank_tag.startswith('R'):
# 		return wordnet.ADV
# 	else:
# 		return ''

#Setting up tokenizer
if(verboseMode): print("Setting up tokenizer.");

tokenizerInput = {
	"stopSet":stopSet,
	"punctuation":punctuation,
	"tokenizer":sent_tokenizer,
	"lematizer":lmtzr,
	"sent_tokenize": sent_tokenize,
	"replaceDictionary": replaceDictionary,
	"ignoreSet":ignoreSet
}

def tokenizeString(theString,maximumTokenSize,tokenizerInput,removeStopWords=True):
	stopSet = tokenizerInput["stopSet"];
	lematizer = tokenizerInput["lematizer"];
	tokenizer = tokenizerInput["tokenizer"];
	punctuation = tokenizerInput["punctuation"];
	sent_tokenize = tokenizerInput["sent_tokenize"];
	replaceDictionary = tokenizerInput["replaceDictionary"];
	ignoreSet = tokenizerInput["ignoreSet"];
	wordsList = [];
	titleAbstract = (". ".join(theString.split("::"))).strip();
	wordsSentences = [word_tokenize(t) for t in sent_tokenize(titleAbstract)];
	stopSentence = False;
	for si, words in enumerate(wordsSentences):
		wordsTags = nltk.pos_tag(words);
		if(stopSentence):
			break;
		for wi,wordTag in enumerate(wordsTags):
			word = wordTag[0];
			tag = wordTag[1];
			
			if word.isdigit() or word[1:].isdigit():
				continue;
# 			if(si>len(wordsSentences)-4 and (word.lower()=="copyright" or (wi>0 and word.lower()=="c" and words[wi-1] == "("  and words[wi+1] == ")" ))):
# 				stopSentence = True;
# 				break;
			word = punctuation.sub("", word);
			convTag = get_wordnet_pos(tag);
			#print "w: "+word;
			if convTag == -1:
				continue
			if(convTag != ''):
				word  = lematizer.lemmatize(word.lower(), convTag);
			else:
				word  = lematizer.lemmatize(word.lower());
			if(len(word)==0 or ((word in stopSet) and removeStopWords) or (word in ignoreSet)):
				continue;
			else:
				if(word in replaceDictionary):
					wordsList.append(replaceDictionary[word]);
				else:
					wordsList.append(word);

	tokens = [set() for i in range(maximumTokenSize)];
	for wordIndex in range(len(wordsList)):
		for tokenSize in range(0,min(wordIndex+1,maximumTokenSize)):
			tokens[tokenSize].add(" ".join(wordsList[(wordIndex-tokenSize):(wordIndex+1)]));
	return tokens;

{'yourself', 'until', 'against', 'again', 'we', 'because', 'were', 'why', 'as', 'own', 'ain', "mustn't", 'which', 'wouldn', 'hers', 'into', 'nor', 'between', 'mustn', 'more', "hasn't", 'who', 'under', 'through', 'an', 'or', "you're", 'itself', 'to', 'hasn', 'both', 'before', 'over', 'only', "won't", 'too', 'not', "you'll", 's', "hadn't", 'during', 'be', 'her', 'she', 'for', 'while', 'off', 'whom', 'very', 'how', 'is', 'so', 'was', 'd', 'few', 'if', 'but', 'there', "doesn't", 'haven', "aren't", 'than', 'no', 'once', 'yourselves', 'further', 'of', 'being', 'o', 'up', "wasn't", "needn't", 'will', 'y', 'below', "you'd", 'these', "mightn't", 'shan', "she's", 'about', 'theirs', 'ourselves', 'all', 'those', "didn't", "wouldn't", 'did', 'doing', 'then', 'that', 'had', 'themselves', 't', 'shouldn', 'what', "shan't", 'our', 'don', 'each', 'won', 'herself', 'am', "couldn't", 'the', 'mightn', 'ma', 'his', 'this', "should've", 'their', 'weren', 'where', 'hadn', 'are', 'you', 'myself', 'they', "don'

In [13]:
# couplingNetwork.vs["Year"] = CI2Year Talvez
import glob
# files = glob.glob('subset_chu/citation_net_*180122.xnet')
# files = glob.glob('subset_chu/citation_net_*%s.xnet' % dateinput)
files = glob.glob('subset_chu/citation_net_*011122.xnet')
for f in files:
    print(f)
files = ['subset_chu/citation_net_11journal_09112022.xnet']

subset_chu/citation_net_acs appl. mater. interfaces_011122.xnet
subset_chu/citation_net_acs nano_011122.xnet
subset_chu/citation_net_adv. funct. mater._011122.xnet
subset_chu/citation_net_adv. mater._011122.xnet
subset_chu/citation_net_chem. mat._011122.xnet
subset_chu/citation_net_j. am. chem. soc._011122.xnet
subset_chu/citation_net_j. mat. chem. b_011122.xnet
subset_chu/citation_net_j. mater. chem. a_011122.xnet
subset_chu/citation_net_j. mater. chem. c_011122.xnet
subset_chu/citation_net_j. mater. chem._011122.xnet
subset_chu/citation_net_langmuir_011122.xnet
subset_chu/citation_net_macromolecules_011122.xnet
subset_chu/citation_net_nano lett._011122.xnet
subset_chu/citation_net_nat. mater._011122.xnet
subset_chu/citation_net_nat. nanotechnol._011122.xnet


In [14]:
# Some util functions

for file in files:

    jsonFileprefix = file[:-5] + '_bardo'
    removeStopWords = True;
    maximumTokenSize = 3; #n-gram
    minKeywordsPerCluster = 10;
    maxKeywordsPerCluster = 10;
    maxClusterNameLength = 150;
    useMajorComponent = True;
    verboseMode = True;

    network = xn.xnet2igraph(file)
    network.vs['wos_id'] = network.vs['name']
    network.vs['name'] = network.vs['title']
    
    
    # Obtaining the major connected component (if needed)
    if(useMajorComponent):
        if(verboseMode): print("Obtaining the major connected component.");
        network = network.clusters("WEAK").giant();

    # Tokenizing the abstracts
    if(verboseMode): print("Tokenizing the abstracts.");
    tokensFrequency = [[] for i in range(maximumTokenSize)];
    tokensGroupFrequency = [{} for i in range(maximumTokenSize)];

    propertiesKeys = set();

    verticesTokens = [];
    for vertexIndex in range(network.vcount()):
        if(vertexIndex%100==0):
            print("Tokenizing: %d/%d             "%(vertexIndex,network.vcount()),end="\r")

    #         for wordsList in tokenList:
    #             tokens = [set() for i in range(maximumTokenSize)];
    #             for wordIndex in range(len(wordsList)):
    #                 for tokenSize in range(0,min(wordIndex+1,maximumTokenSize)):
    #                     tokens[tokenSize].add(" ".join(wordsList[(wordIndex-tokenSize):(wordIndex+1)]));
        verticesTokens.append(tokenizeString(network.vs[vertexIndex]["title"],maximumTokenSize,tokenizerInput));

    print("Done                   ");

    # Obtaining the network community structure
    if(verboseMode): print("Obtaining the network community structure.");
    

    edgelist = [(e.source,e.target) for e in network.es]
    communities = infomapApply(network)[0]
    communities = [int(c) for c in communities]
    print(type(communities[0]), max(communities), min(communities), len(communities))
    print()
    
    
    
    # print("Modularity: %f"%cc.q);

    clusters = [[] for i in range(max(communities)+1)];
    for vertexIndex in range(network.vcount()):
        clusters[communities[vertexIndex]].append(vertexIndex);

    #sorting the clusters by size
    clusters = sorted(clusters,key=len,reverse=True);

    # Getting tokens frequency
    if(verboseMode): print("Getting tokens frequency.");




    tokenFrequencyInClusters = [];
    tokenFrequencyInCorpus = {};

    for clusterIndex in range(len(clusters)):
        cluster = clusters[clusterIndex];
        tokenFrequencyInCluster = {};
        for vertexIndex in cluster:
            tokens = verticesTokens[vertexIndex];
            for tokenSize in range(0,maximumTokenSize):
                for token in tokens[tokenSize]:
                    if(token not in tokenFrequencyInCorpus):
                        tokenFrequencyInCorpus[token] = 0;
                    if(token not in tokenFrequencyInCluster):
                        tokenFrequencyInCluster[token] = 0;
                    tokenFrequencyInCorpus[token] += 1;
                    tokenFrequencyInCluster[token] += 1;
        tokenFrequencyInClusters.append(tokenFrequencyInCluster);

    # Calculating the importance Index
    if(verboseMode): print("Calculating the importance Index.");
    #tokenRelativeFrequencyInClusters = [];
    #tokenRelativeFrequencyOutClusters = [];
    tokenImportanceIndexInClusters = [];

    verticesCount = network.vcount();
    for clusterIndex in range(len(clusters)):
        clusterSize = len(clusters[clusterIndex]);

        tokenFrequencyInCluster = tokenFrequencyInClusters[clusterIndex];

        #tokenRelativeFrequencyInCluster = {};
        #tokenRelativeFrequencyOutCluster = {};
        tokenImportanceIndexInCluster = {};

        for token in tokenFrequencyInCluster:
            nInCluster = tokenFrequencyInCluster[token];
            nOutCluster = tokenFrequencyInCorpus[token]-nInCluster;
            outClusterSize = verticesCount-clusterSize;
            if(nOutCluster==0):
                outClusterSize = 1; #Fix for singletons
            FInCluster = float(nInCluster)/float(clusterSize);
            FOutCluster = float(nOutCluster)/float(outClusterSize);
            importanceIndex = FInCluster-FOutCluster;
            #tokenRelativeFrequencyInCluster[token] = FInCluster;
            #tokenRelativeFrequencyOutCluster[token] = FOutCluster;
            tokenImportanceIndexInCluster[token] = importanceIndex;

        #tokenRelativeFrequencyInClusters.append(tokenRelativeFrequencyInCluster);
        #tokenRelativeFrequencyOutClusters.append(tokenRelativeFrequencyOutCluster);
        tokenImportanceIndexInClusters.append(tokenImportanceIndexInCluster);

    defaultNames = "ABCDEFGHIJKLMNOPQRSTUWVXYZ";
    defaultNamesLength = len(defaultNames);

    clusterKeywords = [];
    minClusterSize = min([len(cluster) for cluster in clusters]);
    maxClusterSize = max([len(cluster) for cluster in clusters]);
    clusterNames = [];
    for clusterIndex in range(len(clusters)):
        cluster = clusters[clusterIndex];
        clusterSize = len(cluster);
        keywords = [v[0] for v in sorted(tokenImportanceIndexInClusters[clusterIndex].items(),key=operator.itemgetter(1),reverse=True)];
        if(maxClusterSize>minClusterSize):
            m = (maxKeywordsPerCluster-minKeywordsPerCluster)/float(maxClusterSize-minClusterSize);
        else:
            m=0;
        keywordsCount = round(m*(clusterSize-minClusterSize)+minKeywordsPerCluster);
        currentKeywords = [];
        while(len(currentKeywords)<keywordsCount and len(keywords)>len(currentKeywords)):
            currentKeywords = keywords[0:keywordsCount];
            jointKeywords = "."+".".join(currentKeywords)+".";
            toRemoveKeywords = [];
            for keyword in currentKeywords:
                if(jointKeywords.find(" %s."%keyword)>=0):
                    toRemoveKeywords.append(keyword);
                elif(jointKeywords.find(".%s "%keyword)>=0):
                    toRemoveKeywords.append(keyword);
            for toRemoveKeyword in toRemoveKeywords:
                keywords.remove(toRemoveKeyword);
                currentKeywords.remove(toRemoveKeyword);
        clusterKeywords.append(currentKeywords);
        #print(currentKeywords);
        clusterName = "";
        if(clusterIndex<defaultNamesLength):
            clusterName += defaultNames[clusterIndex];
        else:
            clusterName += "{%d}"%(clusterIndex);
        clusterName += " - "+", ".join(currentKeywords);
        if(len(clusterName)>maxClusterNameLength):
            clusterName = clusterName[0:maxClusterNameLength-1]+"...";
        for vertexIndex in cluster:
            network.vs[vertexIndex]["Cluster Name"] = clusterName;
            network.vs[vertexIndex]["Cluster Index"] = clusterIndex;
        clusterNames.append(clusterName);
        print(clusterName);


    # Saving the network
    if(verboseMode): print("Saving the network.");
    # network.vs["kcore"] = network.coreness()

    xn.igraph2xnet(network,fileName=PJ('',"%s_infomap_%s.xnet"%(jsonFileprefix, dateoutput)),ignoredNodeAtts=["Text"]);

Obtaining the major connected component.
Tokenizing the abstracts.
Done                   26             
Obtaining the network community structure.
0
1
2
3
<class 'int'> 15 1 132926

Getting tokens frequency.
Calculating the importance Index.
A - nanoparticles, delivery, therapy, cancer, drug, imaging, tumor, release, protein, cell
B - battery, carbon, anode, lithium, cathode, storage, oxygen, reaction, electrode, lithium-ion
C - carbon nanotube, flexible, sensor, composite, membrane, hydrogel, transparent, graphene, film, liquid
D - polymer, perovskite solar cell, transistor, efficient, efficiency, organic solar cell, layer, light-emitting diode, base, device
E - graphene, thermoelectric, mos2, monolayer, two-dimensional, 2d, van der waals, heterostructures, transistor, boron
F - photocatalytic, dye-sensitized solar cell, water, tio2, photoelectrochemical, efficient, enhanced, hydrogen, activity, light
G - plasmonic, gold, optical, plasmon, sers, surface-enhanced raman, resonance, ar

In [30]:
len(set(communities[0]))

45

1191680